In [13]:
import requests

def get_shortest_distance(start_coords, end_coords):
    # Construct the URL
    url = f"http://router.project-osrm.org/route/v1/driving/{start_coords[0]},{start_coords[1]};{end_coords[0]},{end_coords[1]}?overview=false"
    
    # Make the request
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        distance = data['routes'][0]['distance'] / 1000  # Convert to km
        duration = data['routes'][0]['duration'] / 60    # Convert to minutes
        return distance, duration
    else:
        return None, None

# Example coordinates (longitude, latitude)
start_coords = (77.673, 27.579)  # GLA University, Mathura
end_coords = (77.102, 28.704)    # Delhi

# Get the shortest distance and duration
distance, duration = get_shortest_distance(start_coords, end_coords)

if distance is not None:
    print(f"The shortest distance from {start_coords} to {end_coords} is {distance:.2f} km, and it will take approximately {duration:.2f} minutes.")
else:
    print("Failed to retrieve the distance.")


The shortest distance from (77.673, 27.579) to (77.102, 28.704) is 158.44 km, and it will take approximately 126.07 minutes.


In [14]:
def get_distance_and_route(ors_api_key, start_coordinates, end_coordinates):
    """
    Calculate the distance and route using OpenRouteService API.
    """
    url = "https://api.openrouteservice.org/v2/directions/driving-car"
    data = {
        "coordinates": [
            [start_coordinates[1], start_coordinates[0]],  # [Longitude, Latitude]
            [end_coordinates[1], end_coordinates[0]]
        ]
    }

    response = requests.post(url, json=data, headers={"Authorization": ors_api_key})

    # Print the response for debugging
    print("API Response:", response.json())

    if response.status_code == 200:
        try:
            data = response.json()
            if isinstance(data, dict) and "routes" in data and len(data["routes"]) > 0:
                route = data["routes"][0]
                distance = route["summary"]["distance"]
                route_coordinates = route["geometry"]["coordinates"]  # Get the coordinates of the route
                return distance, route_coordinates
            else:
                print("No valid routes found in the response.")
                return None, None
        except (KeyError, IndexError) as e:
            print("Error parsing route data:", e)
            return None, None
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None, None


In [15]:
import requests
from geopy.geocoders import Nominatim
import time

def geocode_city(city_name):
    """Get the coordinates of a city using its name."""
    geolocator = Nominatim(user_agent="MyGeocoder (guptavarun132@gmail.com)")  # Update user agent
    location = geolocator.geocode(city_name)
    if location:
        return (location.longitude, location.latitude)
    else:
        print(f"Could not find coordinates for {city_name}.")
        return None

def get_shortest_distance(start_coords, end_coords):
    """Get the shortest distance and duration between two coordinates."""
    url = f"http://router.project-osrm.org/route/v1/driving/{start_coords[0]},{start_coords[1]};{end_coords[0]},{end_coords[1]}?overview=false"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        distance = data['routes'][0]['distance'] / 1000  # Convert to kilometers
        duration = data['routes'][0]['duration'] / 60    # Convert to minutes
        return distance, duration
    else:
        print(f"Error retrieving distance: {response.status_code}")
        return None, None

# Example city names
start_city = "Mathura, India"
end_city = "Delhi, India"

# Geocode the cities
start_coords = geocode_city(start_city)
time.sleep(1)  # Respect rate limits
end_coords = geocode_city(end_city)

if start_coords and end_coords:
    # Get the shortest distance and duration
    distance, duration = get_shortest_distance(start_coords, end_coords)

    if distance is not None:
        print(f"The shortest distance from {start_city} to {end_city} is {distance:.2f} km, and it will take approximately {duration:.2f} minutes.")
    else:
        print("Failed to retrieve the distance.")
else:
    print("Could not geocode one or both cities.")


The shortest distance from Mathura, India to Delhi, India is 133.66 km, and it will take approximately 106.34 minutes.


In [16]:
import requests
from geopy.geocoders import Nominatim
from geopy.distance import great_circle

def get_coordinates(location_name):
    """Get latitude and longitude for a given location name."""
    geolocator = Nominatim(user_agent="geoapiExercises")
    try:
        location = geolocator.geocode(location_name)
        if location:
            return (location.latitude, location.longitude)
        else:
            print(f"Could not find coordinates for {location_name}.")
            return None
    except Exception as e:
        print(f"Error retrieving coordinates: {e}")
        return None

def get_distance(loc1, loc2, mode='driving'):
    """Calculate the distance between two locations."""
    distance = great_circle(loc1, loc2).km
    if mode == 'driving':
        speed = 60  # average speed in km/h
    elif mode == 'walking':
        speed = 5   # average speed in km/h
    elif mode == 'bicycling':
        speed = 15  # average speed in km/h
    else:
        speed = 60  # default to driving speed

    time_minutes = (distance / speed) * 60
    return distance, time_minutes

# Example usage
place1 = "Mathura, India"
place2 = "Delhi, India"

coordinates1 = get_coordinates(place1)
coordinates2 = get_coordinates(place2)

if coordinates1 and coordinates2:
    distance, time = get_distance(coordinates1, coordinates2)
    print(f"The shortest distance from {place1} to {place2} is {distance:.2f} km, and it will take approximately {time:.2f} minutes.")
else:
    print("Could not geocode one or both cities.")



Error retrieving coordinates: Non-successful status code 403
Error retrieving coordinates: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Delhi%2C+India&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Could not geocode one or both cities.


In [17]:
import requests

def get_distance(api_key, start_coordinates, end_coordinates):
    """
    Calculate the distance between two places using the OpenRouteService API.
    
    Parameters:
    - api_key: Your OpenRouteService API key.
    - start_coordinates: Tuple of (latitude, longitude) for the starting location.
    - end_coordinates: Tuple of (latitude, longitude) for the ending location.
    
    Returns:
    - Distance in meters, or None if the request fails.
    """
    url = "https://api.openrouteservice.org/v2/directions/driving-car"

    # Data to be sent in POST request
    data = {
        "coordinates": [
            [start_coordinates[1], start_coordinates[0]],  # [Longitude, Latitude]
            [end_coordinates[1], end_coordinates[0]]
        ]
    }

    # Making the request
    response = requests.post(url, json=data, headers={"Authorization": api_key})

    if response.status_code == 200:
        # Parsing the response JSON to get the distance
        route = response.json()["routes"][0]
        distance = route["summary"]["distance"]
        return distance
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

# Example usage
api_key = "5b3ce3597851110001cf6248d1ea735cae4d4df4addf04b3c296117d"  # Replace with your actual API key
start_coordinates = (52.5200, 13.4050)  # Example: Berlin, Germany
end_coordinates = (48.8566, 2.3522)  # Example: Paris, France

distance = get_distance(api_key, start_coordinates, end_coordinates)
if distance is not None:
    print(f"The distance between the two locations is {distance / 1000} kilometers.")
else:
    print("Failed to get distance.")


The distance between the two locations is 1048.3843 kilometers.


In [18]:
import requests
from geopy.geocoders import Nominatim

def geocode_city(city_name):
    geolocator = Nominatim(user_agent="geo_distance_calculator")
    location = geolocator.geocode(city_name)

    if location:
        return (location.latitude, location.longitude)
    else:
        print(f"Could not find location for city: {city_name}")
        return None

def get_distance_and_time(ors_api_key, start_coordinates, end_coordinates):
    url = "https://api.openrouteservice.org/v2/directions/driving-car"

    data = {
        "coordinates": [
            [start_coordinates[1], start_coordinates[0]],  # [Longitude, Latitude]
            [end_coordinates[1], end_coordinates[0]]
        ]
    }

    response = requests.post(url, json=data, headers={"Authorization": ors_api_key})

    if response.status_code == 200:
        route = response.json()["routes"][0]
        distance = route["summary"]["distance"]  # Distance in meters
        duration = route["summary"]["duration"]  # Duration in seconds
        return distance, duration
    else:
        print(f"Error: {response.status_code} - {response.json()}")
        return None, None

# Example usage
ors_api_key = "5b3ce3597851110001cf6248d1ea735cae4d4df4addf04b3c296117d"  # Your API key
start_city = "Mathura, India"  # Starting city
end_city = "Khajuraho, India"  # Destination city

# Geocode the cities to get coordinates
start_coordinates = geocode_city(start_city)
end_coordinates = geocode_city(end_city)

# Debug output
print(f"Start coordinates: {start_coordinates}")
print(f"End coordinates: {end_coordinates}")

if start_coordinates and end_coordinates:
    distance, duration = get_distance_and_time(ors_api_key, start_coordinates, end_coordinates)
    if distance is not None and duration is not None:
        distance_km = distance / 1000  # Convert meters to kilometers
        duration_hours = duration / 3600  # Convert seconds to hours
        print(f"The distance between {start_city} and {end_city} is {distance_km:.2f} kilometers.")
        print(f"The estimated travel time is {duration_hours:.2f} hours.")  # Convert seconds to hours
        
        # Calculate required speed
        required_speed = distance_km / duration_hours  # Speed in km/h
        print(f"You need to maintain a speed of {required_speed:.2f} km/h to reach your destination on time.")
    else:
        print("Failed to get distance or duration.")
else:
    print("Failed to geocode one or both cities.")


Start coordinates: (27.633333, 77.583333)
End coordinates: (24.8515132, 79.9259786)
The distance between Mathura, India and Khajuraho, India is 543.04 kilometers.
The estimated travel time is 6.15 hours.
You need to maintain a speed of 88.28 km/h to reach your destination on time.


In [19]:
import requests
from geopy.geocoders import Nominatim

def geocode_city(city_name):
    geolocator = Nominatim(user_agent="geo_distance_calculator")
    location = geolocator.geocode(city_name)

    if location:
        return (location.latitude, location.longitude)
    else:
        print(f"Could not find location for city: {city_name}")
        return None

def get_distance_and_time(ors_api_key, start_coordinates, end_coordinates):
    url = "https://api.openrouteservice.org/v2/directions/driving-car"

    data = {
        "coordinates": [
            [start_coordinates[1], start_coordinates[0]],  # [Longitude, Latitude]
            [end_coordinates[1], end_coordinates[0]]
        ]
    }

    response = requests.post(url, json=data, headers={"Authorization": ors_api_key})

    if response.status_code == 200:
        route = response.json()["routes"][0]
        distance = route["summary"]["distance"]  # Distance in meters
        duration = route["summary"]["duration"]  # Duration in seconds
        return distance, duration
    else:
        print(f"Error: {response.status_code} - {response.json()}")
        return None, None

# Example usage
ors_api_key = "5b3ce3597851110001cf6248d1ea735cae4d4df4addf04b3c296117d"  # Your API key
start_city = "Mathura, India"  # Starting city
end_city = "Khajuraho, India"  # Destination city

# Geocode the cities to get coordinates
start_coordinates = geocode_city(start_city)
end_coordinates = geocode_city(end_city)

# Debug output
print(f"Start coordinates: {start_coordinates}")
print(f"End coordinates: {end_coordinates}")

if start_coordinates and end_coordinates:
    distance, duration = get_distance_and_time(ors_api_key, start_coordinates, end_coordinates)
    if distance is not None and duration is not None:
        print(f"The distance between {start_city} and {end_city} is {distance / 1000} kilometers.")
        print(f"The estimated travel time is {duration / 3600} hours.")  # Convert seconds to hours
    else:
        print("Failed to get distance or duration.")
else:
    print("Failed to geocode one or both cities.")


Start coordinates: (27.633333, 77.583333)
End coordinates: (24.8515132, 79.9259786)
The distance between Mathura, India and Khajuraho, India is 543.0427 kilometers.
The estimated travel time is 6.151694444444444 hours.


In [20]:
import requests
from geopy.geocoders import Nominatim

def geocode_city(city_name):
    geolocator = Nominatim(user_agent="geo_distance_calculator")
    location = geolocator.geocode(city_name)

    if location:
        return (location.latitude, location.longitude)
    else:
        print(f"Could not find location for city: {city_name}")
        return None

def get_distance_and_time(ors_api_key, start_coordinates, end_coordinates):
    url = "https://api.openrouteservice.org/v2/directions/driving-car"

    data = {
        "coordinates": [
            [start_coordinates[1], start_coordinates[0]],  # [Longitude, Latitude]
            [end_coordinates[1], end_coordinates[0]]
        ]
    }

    response = requests.post(url, json=data, headers={"Authorization": ors_api_key})

    if response.status_code == 200:
        route = response.json()["routes"][0]
        distance = route["summary"]["distance"]  # Distance in meters
        duration = route["summary"]["duration"]  # Duration in seconds
        return distance, duration
    else:
        print(f"Error: {response.status_code} - {response.json()}")
        return None, None

# Example usage
ors_api_key = "5b3ce3597851110001cf6248d1ea735cae4d4df4addf04b3c296117d"  # Your API key
start_city = "Mathura, India"  # Starting city
end_city = "Khajuraho, India"  # Destination city

# Geocode the cities to get coordinates
start_coordinates = geocode_city(start_city)
end_coordinates = geocode_city(end_city)

# Debug output
print(f"Start coordinates: {start_coordinates}")
print(f"End coordinates: {end_coordinates}")

if start_coordinates and end_coordinates:
    distance, duration = get_distance_and_time(ors_api_key, start_coordinates, end_coordinates)
    if distance is not None and duration is not None:
        print(f"The distance between {start_city} and {end_city} is {distance / 1000} kilometers.")
        print(f"The estimated travel time is {duration / 60} minutes.")  # Convert seconds to minutes
    else:
        print("Failed to get distance or duration.")
else:
    print("Failed to geocode one or both cities.")


Start coordinates: (27.633333, 77.583333)
End coordinates: (24.8515132, 79.9259786)
The distance between Mathura, India and Khajuraho, India is 543.0427 kilometers.
The estimated travel time is 369.10166666666663 minutes.


Start coordinates: (27.633333, 77.583333)
End coordinates: (24.8515132, 79.9259786)
The distance between Mathura, India and Khajuraho, India is 543.0427 kilometers.
